In [ ]:
!pip install aixplain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 3.1.6
    Uninstalling Jinja2-3.1.6:
      Successfully uninstalled Jinja2-3.1.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is

In [ ]:
def new(game_id: str=""):
    """
    Starts a new Blackjack game.
    Shuffles the deck and deals two cards each to the player and dealer.
    Prints the current state of the game, including the player's and dealer's cards and scores, and the game ID.

    Parameters:
    game_id (str): The ID of the game deck. It must be a valid ID. Set game_id to an empty string ("") to use a new deck.
    """

    import requests, textwrap

    if game_id:
        deck_id = game_id
        requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/shuffle/").raise_for_status()
    else:
        response = requests.get(f"https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1")
        response.raise_for_status()
        deck_id = response.json()['deck_id']

    count = 2

    # Player cards
    response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count={count}")
    response.raise_for_status()
    player_cards = response.json()['cards']

    # Dealer cards
    response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count={count}")
    response.raise_for_status()
    dealer_cards = response.json()['cards']

    dealer_visible_cards = dealer_cards[:1]
    requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/player/add/?cards={','.join([card['code'] for card in player_cards])}").raise_for_status()
    requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/dealer/add/?cards={','.join([card['code'] for card in dealer_cards])}").raise_for_status()

    # Calculate player score
    player_score = 0
    player_aces = 0
    for card in player_cards:
        if card['value'] in ['JACK', 'QUEEN', 'KING']:
            player_score += 10
        elif card['value'] == 'ACE':
            player_aces += 1
            player_score += 11
        else:
            player_score += int(card['value'])
    while player_score > 21 and player_aces:
        player_score -= 10
        player_aces -= 1

    # Calculate dealer visible score
    dealer_visible_score = 0
    dealer_visible_aces = 0
    for card in dealer_visible_cards:
        if card['value'] in ['JACK', 'QUEEN', 'KING']:
            dealer_visible_score += 10
        elif card['value'] == 'ACE':
            dealer_visible_aces += 1
            dealer_visible_score += 11
        else:
            dealer_visible_score += int(card['value'])
    while dealer_visible_score > 21 and dealer_visible_aces:
        dealer_visible_score -= 10
        dealer_visible_aces -= 1

    output = textwrap.dedent(f"""
    Player's cards: {[card['code'] for card in player_cards]}
    Dealer's visible cards: {[card['code'] for card in dealer_visible_cards]}
    Player's score: {player_score}
    Dealer's visible score: {dealer_visible_score}
    Game ID: {deck_id}
    """)

    return output

def hit(game_id: str):
    """
    Draws a new card for the player and updates the player's score.
    If the player's score exceeds 21, the player busts.
    Prints the current state of the game, including the player's cards and score, and the game ID.

    Parameters:
    game_id (str): The ID of the game deck.
    """

    import requests, textwrap

    deck_id = game_id

    # Draw a new card
    response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1")
    response.raise_for_status()
    new_card = response.json()['cards']

    # Add to player's pile
    requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/player/add/?cards={','.join([card['code'] for card in new_card])}").raise_for_status()

    # List player's pile
    response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/player/list/")
    response.raise_for_status()
    player_pile = response.json()['piles']['player']['cards']

    # Calculate player score
    player_score = 0
    player_aces = 0
    for card in player_pile:
        if card['value'] in ['JACK', 'QUEEN', 'KING']:
            player_score += 10
        elif card['value'] == 'ACE':
            player_aces += 1
            player_score += 11
        else:
            player_score += int(card['value'])
    while player_score > 21 and player_aces:
        player_score -= 10
        player_aces -= 1


    output = textwrap.dedent(f"""
    Player drew: {[card['code'] for card in new_card]}
    Player's cards: {[card['code'] for card in player_pile]}
    Player's score: {player_score}
    Game ID: {deck_id}
    """)

    if player_score > 21:
        output += "\nPlayer busts! Dealer wins."

    return output

def stand(game_id: str):
    """
    Ends the player's turn and reveals the dealer's cards.
    The dealer draws cards until their score is 17 or higher.
    Prints the final state of the game, including the player's and dealer's cards and scores, and the game ID.

    Parameters:
    game_id (str): The ID of the game deck.
    """

    import requests, textwrap

    deck_id = game_id

    # List player's pile
    response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/player/list/")
    response.raise_for_status()
    player_pile = response.json()['piles']['player']['cards']

    # List dealer's pile
    response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/dealer/list/")
    response.raise_for_status()
    dealer_pile = response.json()['piles']['dealer']['cards']

    # Calculate player score
    player_score = 0
    player_aces = 0
    for card in player_pile:
        if card['value'] in ['JACK', 'QUEEN', 'KING']:
            player_score += 10
        elif card['value'] == 'ACE':
            player_aces += 1
            player_score += 11
        else:
            player_score += int(card['value'])
    while player_score > 21 and player_aces:
        player_score -= 10
        player_aces -= 1

    # Calculate dealer score
    dealer_score = 0
    dealer_aces = 0
    for card in dealer_pile:
        if card['value'] in ['JACK', 'QUEEN', 'KING']:
            dealer_score += 10
        elif card['value'] == 'ACE':
            dealer_aces += 1
            dealer_score += 11
        else:
            dealer_score += int(card['value'])
    while dealer_score > 21 and dealer_aces:
        dealer_score -= 10
        dealer_aces -= 1

    while dealer_score < 17:
        response = requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1")
        response.raise_for_status()
        new_card = response.json()['cards']
        requests.get(f"https://deckofcardsapi.com/api/deck/{deck_id}/pile/dealer/add/?cards={','.join([card['code'] for card in new_card])}").raise_for_status()
        dealer_pile.append(new_card[0])
        dealer_score = 0
        dealer_aces = 0
        for card in dealer_pile:
            if card['value'] in ['JACK', 'QUEEN', 'KING']:
                dealer_score += 10
            elif card['value'] == 'ACE':
                dealer_aces += 1
                dealer_score += 11
            else:
                dealer_score += int(card['value'])
        while dealer_score > 21 and dealer_aces:
            dealer_score -= 10
            dealer_aces -= 1

    output = textwrap.dedent(f"""
        Player's cards: {[card['code'] for card in player_pile]}
        Player's score: {player_score}
        Dealer's cards: {[card['code'] for card in dealer_pile]}
        Dealer's score: {dealer_score}
        """)

    if dealer_score > 21 or player_score > dealer_score:
        output += "Player wins!\n"
    elif player_score < dealer_score:
        output += "Dealer wins!\n"
    else:
        output += "It's a tie!\n"

    output += f"Game ID: {deck_id}"

    return output

    return output

In [ ]:
print(new())



Player's cards: ['3S', '7S']
Dealer's visible cards: ['KD']
Player's score: 10
Dealer's visible score: 10
Game ID: bs6lb3kqny9b



In [ ]:
print(hit("bs6lb3kqny9b"))



Player drew: ['7C']
Player's cards: ['3S', '7S', '7C']
Player's score: 17
Game ID: bs6lb3kqny9b



In [ ]:
print(stand("bs6lb3kqny9b"))


Player's cards: ['3S', '7S', '7C']
Player's score: 17
Dealer's cards: ['KD', '7H']
Dealer's score: 17
It's a tie!
Game ID: bs6lb3kqny9b


In [ ]:
import json
from aixplain import Aixplain
aix = Aixplain("7604245cb41d746d755bbf5b9357c6294503f100472adf8832ba5891b9a858f6") # aixplain team
# aix = Aixplain("8e6e248e8262660c0c8334769fbd48750899b1a389463d6ba93f0a20dbdcf7f5") # Tim Nelson team

In [ ]:
model_list = aix.Model.list(query="Blackjack")['results']
for model in model_list:
    print(model.id, model.name)

67e18e564fa177b0d6cd1663 Blackjack New
67d3493902912431287a389e Blackjack Hit
67d3493a8e9326b58bc20ad0 Blackjack Stand


In [ ]:
blackjack_new = aix.Model.get("67d36a128e9326b58bc20ddc")
blackjack_new.delete()

In [ ]:
blackjack_hit = aix.Model.get("67d3493902912431287a389e")
blackjack_hit.delete()

In [ ]:
blackjack_stand = aix.Model.get("67d3493a8e9326b58bc20ad0")
blackjack_stand.delete()

In [ ]:
blackjack_new = aix.Model.create_utility_model(
    name="Blackjack New",
    code=new,
)

In [ ]:
blackjack_hit = aix.Model.create_utility_model(
    name="Blackjack Hit",
    code=hit,
)

In [ ]:
blackjack_stand = aix.Model.create_utility_model(
    name="Blackjack Stand",
    code=stand,
)

In [ ]:
response = blackjack_new.run("")
print(response.data)

Player's cards: ['8H', 'AC']
Dealer's visible cards: ['9H']
Player's score: 19
Dealer's visible score: 9
Game ID: 1wred04rp03l


In [ ]:
response = blackjack_hit.run("1wred04rp03l")
print(response.data)

Player drew: ['0D']
Player's cards: ['7H', '3S', '0D']
Player's score: 20
Game ID: iteet1lwbzvk


In [ ]:
response = blackjack_stand.run("1wred04rp03l")
print(response.data)

Player's cards: ['8H', 'AC']
Player's score: 19
Dealer's cards: ['9H', '8S']
Dealer's score: 17
Player wins!
Game ID: 1wred04rp03l


In [ ]:
blackjack_agent = aix.Agent.create(
    name="Blackjack Agent",
    description="An agent that can teach and play the game of Blackjack.",
    instructions="""
    You are an agent that can teach and play the game of Blackjack.

    To play Blackjack with the player, use the Blackjack utilities:
    Blackjack New: Starts a new game of Blackjack.
    Blackjack Hit: Draws a new card for the player.
    Blackjack Stand: Ends the player's turn and plays out the dealer's hand.

    You must pass the game ID when using Blackjack Hit and Blackjack Stand.
    You must pass an empty string ("") as the game ID when using Blackjack New to start a new game.
    However, you can pass the game ID returned by Blackjack New to restart the same game.

    The player does not have to specify the game ID. Look for it in the conversation history.

    The game ID is returned by the Blackjack New utility. You must inform the player of their game ID in every response after using Blackjack New, Hit or Stand.

    Note that the Utilities return 10s as 0s. Always refer to 0s as 10s when responding to the player.

    You can play the following variant with the player:

    Number of Decks:
    The game uses a single deck of cards, which is shuffled at the start of each new game.

    Game States:

    1. New: Starts a new game, shuffles the deck, and deals initial cards to the player and dealer.
    2. Hit: Draws a new card for the player and updates the player's score. Checks if the player busts.
    3. Stand: Ends the player's turn, reveals the dealer's cards, and plays out the dealer's hand according to Blackjack rules. Determines the winner based on the final scores.

    Initial Deal:
    1. Both the player and the dealer are dealt two cards each.
    2. The player's cards are both visible.
    3. The dealer's cards include one visible card and one hidden card.

    Player Options:
    1. Hit: The player can choose to draw additional cards to increase their score.
    2. Stand: The player can choose to stop drawing cards and end their turn.

    Scoring:
    1. Number cards (2-10) are worth their face value.
    2. Face cards (Jack, Queen, King) are worth 10 points each.
    3. Aces can be worth either 1 or 11 points, depending on which value is more favorable without causing the player to bust.

    Dealer's Turn:
    1. The dealer reveals their hidden card and continues to draw cards until their score is at least 17.
    2. If the dealer's score exceeds 21, the dealer busts, and the player wins.

    Winning Conditions:
    1. The player wins if their score is higher than the dealer's without exceeding 21.
    2. The dealer wins if their score is higher than the player's without exceeding 21.
    3. If both the player and dealer have the same score, it is a tie.

    Additional Rules Not Part of This Variant
    1. Double Down: In some Blackjack variants, players can double their initial bet after receiving their first two cards and receive exactly one more card. This option is not available in this game.

    2. Split: If the player's first two cards are of the same value, they can choose to split them into two separate hands. This option is not available in this game.

    3. Insurance: If the dealer's visible card is an Ace, players can take insurance, a side bet that the dealer has a Blackjack. This option is not available in this game.

    4. Surrender: Some variants allow players to surrender their hand and lose half their bet instead of playing out the hand. This option is not available in this game.
    Multiple Decks:

    Some Blackjack games use multiple decks (e.g., 6 or 8 decks) to reduce the effectiveness of card counting. This game uses only a single deck.


    """,
    tools=[
        blackjack_new,
        blackjack_hit,
        blackjack_stand,
    ]
)

In [ ]:
session_id=None

In [ ]:
response = blackjack_agent.run("What can you do?")
print(response.data['output'])

I can teach and play the game of Blackjack with you. I can start a new game, draw cards for you, and reveal the dealer's hand to determine the winner.


In [ ]:
response = blackjack_agent.run("What are the rules?", session_id=response.data['session_id'])
print(response.data['output'])

In Blackjack, the goal is to have a hand value closer to 21 than the dealer without going over. Number cards (2-10) are worth their face value, face cards (Jack, Queen, King) are worth 10 points, and Aces can be worth 1 or 11 points. Players can choose to 'Hit' to draw more cards or 'Stand' to end their turn. The dealer must draw until they reach a score of at least 17. If a player's score exceeds 21, they bust and lose the game. The player wins if their score is higher than the dealer's without busting.


In [ ]:
response = blackjack_agent.run("I want to play a new game of Blackjack.", session_id=response.data['session_id'])
print(response.data['output'])

A new game of Blackjack has started! Your cards are 5 of Spades and Jack of Spades, giving you a score of 15. The dealer's visible card is 10. Your game ID is oroorx3cu0o1. Would you like to hit or stand?


In [ ]:
# print(json.dumps(response.data["intermediate_steps"], indent=4))

In [ ]:
response = blackjack_agent.run("I want to hit.", session_id=response.data['session_id'])
print(response.data['output'])

You drew a King of Hearts, bringing your total score to 25. Unfortunately, you bust! The dealer wins.


In [ ]:
response = blackjack_agent.run("I want to stand.", session_id=response.data['session_id'])
print(response.data['output'])

You chose to stand. Your final score is 20, while the dealer's score is 21. Unfortunately, the dealer wins this round. Your game ID is fwrcudldihso.


In [ ]:
# response = blackjack_agent.run("What is my game ID?", session_id=response.data['session_id'])
# print(response.data['output'])

In [ ]:
response = blackjack_agent.run("Let's play again!", session_id=response.data['session_id'])
print(response.data['output'])

A new game of Blackjack has started! Your cards are Ace of Clubs and Ace of Spades, giving you a score of 12. The dealer's visible card is 5 of Diamonds. Your game ID is lebzczwx8338. Would you like to hit or stand?


In [ ]:
response = blackjack_agent.run("I want to hit.", session_id=response.data['session_id'])
print(response.data['output'])

You drew a Jack of Clubs, bringing your total score to 12. Your cards are now Ace of Clubs, Ace of Spades, and Jack of Clubs. The dealer's visible card is still 5 of Diamonds. Would you like to hit or stand?


In [ ]:
response = blackjack_agent.run("Why is my score still the same?", session_id=response.data['session_id'])
print(response.data['output'])

Your score is still 12 because the two Aces can be counted as 1 or 11, and in this case, they are counted as 1 each. The Jack is worth 10, so your total is 1 + 1 + 10 = 12. Would you like to hit or stand?


In [ ]:
response = blackjack_agent.run("What do you recommend I do?", session_id=response.data['session_id'])
print(response.data['output'])

I recommend hitting again to try to improve your score, as you currently have a total of 12.


In [ ]:
response = blackjack_agent.run("Okay, sure, let's hit.", session_id=response.data['session_id'])
print(response.data['output'])

You drew a Jack of Hearts, bringing your total score to 22. Unfortunately, you bust again! The dealer wins.
